In [3]:
import os
import numpy as np
import pandas as pd
import openpyxl
import re

In [4]:
!pip install openpyxl

DEPRECATION: Loading egg at c:\users\adem mcharek\fruits\build\tfod\lib\site-packages\apache_beam-2.47.0rc1-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\adem mcharek\fruits\build\tfod\lib\site-packages\avro_python3-1.10.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\adem mcharek\fruits\build\tfod\lib\site-packages\contextlib2-21.6.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\adem mcharek\fruits\build

In [5]:
filepath = '3-20-24.csv'
data = pd.read_csv(filepath)
data.head()

,Company,Role,Location,Application/Link,Date Posted
0,**[Western Digital](https://simplify.jobs/c/We...,Summer 2024 Software Engineering Intern,"Longmont, CO",<a href=https://jobs.smartrecruiters.com/Weste...,Mar 18
1,**[Vestmark](https://simplify.jobs/c/Vestmark)**,Intern – Software Engineer - Summer 2024,"Wakefield, MA",<a href=https://boards.greenhouse.io/vestmark/...,Mar 18
2,**[Trumid](https://simplify.jobs/c/Trumid)**,Technology Intern,Remote in USA,<a href=https://boards.greenhouse.io/trumid/jo...,Mar 18
3,**[Square](https://simplify.jobs/c/Square)**,Manufacturing Electrical Engineer Intern - 12-...,"Oakland, CA</br>Remote in USA",<a href=https://jobs.smartrecruiters.com/Squar...,Mar 18
4,**[Sensata](https://simplify.jobs/c/sensata-te...,IT – Data Analyst Intern-Summer 2024,"Attleboro, MA",<a href=https://sensata.wd1.myworkdayjobs.com/...,Mar 18


In [6]:
data['Company'] = data.Company.str.extract(r'\[(.*?)\]', expand=False)

data.head()

,Company,Role,Location,Application/Link,Date Posted
0,Western Digital,Summer 2024 Software Engineering Intern,"Longmont, CO",<a href=https://jobs.smartrecruiters.com/Weste...,Mar 18
1,Vestmark,Intern – Software Engineer - Summer 2024,"Wakefield, MA",<a href=https://boards.greenhouse.io/vestmark/...,Mar 18
2,Trumid,Technology Intern,Remote in USA,<a href=https://boards.greenhouse.io/trumid/jo...,Mar 18
3,Square,Manufacturing Electrical Engineer Intern - 12-...,"Oakland, CA</br>Remote in USA",<a href=https://jobs.smartrecruiters.com/Squar...,Mar 18
4,Sensata,IT – Data Analyst Intern-Summer 2024,"Attleboro, MA",<a href=https://sensata.wd1.myworkdayjobs.com/...,Mar 18


In [7]:
data['Application'] = data['Application/Link'].str.extract(r'<a(.*?)a>', expand=False)
data['Application'] = data['Application'].str.replace('href=', '')
data['Application'] = data['Application'].str.extract(r'(https.*Simplify)', expand=False)
data['Application']

0     https://jobs.smartrecruiters.com/WesternDigita...
1     https://boards.greenhouse.io/vestmark/jobs/580...
2     https://boards.greenhouse.io/trumid/jobs/59230...
3     https://jobs.smartrecruiters.com/Square/743999...
4     https://sensata.wd1.myworkdayjobs.com/en-US/Se...
5     https://rockwellautomation.wd1.myworkdayjobs.c...
6     https://jobs.lever.co/postera/70212446-041d-4b...
7     https://leidos.wd5.myworkdayjobs.com/External/...
8     https://leidos.wd5.myworkdayjobs.com/External/...
9     https://leidos.wd5.myworkdayjobs.com/External/...
10    https://leidos.wd5.myworkdayjobs.com/External/...
11    https://dell.wd1.myworkdayjobs.com/External/jo...
12    https://dell.wd1.myworkdayjobs.com/External/jo...
13    https://crowdstrike.wd5.myworkdayjobs.com/en-u...
14    https://www.amazon.jobs/en/jobs/2545578/amazon...
15    https://jobs.smartrecruiters.com/Visa/74399997...
16    https://boards.greenhouse.io/soundcloud71/jobs...
17    https://intel.wd1.myworkdayjobs.com/en-us/

In [8]:
data['Application'][1]

'https://boards.greenhouse.io/vestmark/jobs/5802883?utm_source=Simplify&ref=Simplify'

In [9]:
data['link'] = data['Application/Link'].str.extract(r'(https://simplify.*)')
data['link'] = data['link'].str.replace(r'".*', '"')
data['link'] = data['link'].str.extract(r'(https://[^"\s>]+)', expand=False) 
data['link'][1]

'https://simplify.jobs/p/ff48349d-6c53-409b-8ddc-c6a273140c86?utm_source=GHList'

In [10]:
data['link'][1]

'https://simplify.jobs/p/ff48349d-6c53-409b-8ddc-c6a273140c86?utm_source=GHList'

In [11]:

mask = data['Location'].str.startswith('<')
data.loc[mask, 'Location'] = data.loc[mask, 'Location'].str.extract(r'<\*\*(.*?)\*\*')


In [12]:
data = data.drop('Application/Link', axis=1)


In [13]:
data.head()

,Company,Role,Location,Date Posted,Application,link
0,Western Digital,Summer 2024 Software Engineering Intern,"Longmont, CO",Mar 18,https://jobs.smartrecruiters.com/WesternDigita...,https://simplify.jobs/p/bd4c78d9-1ec8-4abd-9d9...
1,Vestmark,Intern – Software Engineer - Summer 2024,"Wakefield, MA",Mar 18,https://boards.greenhouse.io/vestmark/jobs/580...,https://simplify.jobs/p/ff48349d-6c53-409b-8dd...
2,Trumid,Technology Intern,Remote in USA,Mar 18,https://boards.greenhouse.io/trumid/jobs/59230...,https://simplify.jobs/p/e4af238a-1542-41d4-9de...
3,Square,Manufacturing Electrical Engineer Intern - 12-...,"Oakland, CA</br>Remote in USA",Mar 18,https://jobs.smartrecruiters.com/Square/743999...,https://simplify.jobs/p/a75a6316-7a0b-4c32-a95...
4,Sensata,IT – Data Analyst Intern-Summer 2024,"Attleboro, MA",Mar 18,https://sensata.wd1.myworkdayjobs.com/en-US/Se...,https://simplify.jobs/p/c3f01eea-f583-4039-a91...


In [14]:
data['Company'].fillna(method='ffill', inplace=True)

In [15]:
data['link'].fillna(method='ffill', inplace=True)

In [16]:
data.rename(columns={'Date Posted': 'Date'}, inplace=True)


In [17]:
data.head()

,Company,Role,Location,Date,Application,link
0,Western Digital,Summer 2024 Software Engineering Intern,"Longmont, CO",Mar 18,https://jobs.smartrecruiters.com/WesternDigita...,https://simplify.jobs/p/bd4c78d9-1ec8-4abd-9d9...
1,Vestmark,Intern – Software Engineer - Summer 2024,"Wakefield, MA",Mar 18,https://boards.greenhouse.io/vestmark/jobs/580...,https://simplify.jobs/p/ff48349d-6c53-409b-8dd...
2,Trumid,Technology Intern,Remote in USA,Mar 18,https://boards.greenhouse.io/trumid/jobs/59230...,https://simplify.jobs/p/e4af238a-1542-41d4-9de...
3,Square,Manufacturing Electrical Engineer Intern - 12-...,"Oakland, CA</br>Remote in USA",Mar 18,https://jobs.smartrecruiters.com/Square/743999...,https://simplify.jobs/p/a75a6316-7a0b-4c32-a95...
4,Sensata,IT – Data Analyst Intern-Summer 2024,"Attleboro, MA",Mar 18,https://sensata.wd1.myworkdayjobs.com/en-US/Se...,https://simplify.jobs/p/c3f01eea-f583-4039-a91...


In [15]:
data.to_excel('2-17-24.xlsx', index=False)


In [19]:
data.to_csv('3-20-24-clean.csv', index=False)
